In [42]:
import pandas as pd

data_path = "Documentacion/Data_Entry_2017_with_paths.csv"
data_df = pd.read_csv(data_path)

test_list_path = "Documentacion/test_list.txt"
train_val_list_path = "Documentacion/train_val_list.txt"

with open(test_list_path, 'r') as f:
    test_list = f.read().splitlines()

with open(train_val_list_path, 'r') as f:
    train_val_list = f.read().splitlines()

test_df = data_df[data_df['Image Index'].isin(test_list)]
train_val_df = data_df[data_df['Image Index'].isin(train_val_list)]

# Guardar archivos CSV
test_df.to_csv("Documentacion/test.csv", index=False)
train_val_df.to_csv("Documentacion/train_val.csv", index=False)

print("Archivos de prueba y entrenamiento/validación creados exitosamente.")

labels = ["No Finding", "Infiltration", "Atelectasis", "Effusion", "Nodule", "Pneumothorax", "Mass", "Consolidation", "Pleural_Thickening", "Cardiomegaly", "Emphysema", "Fibrosis", "Edema", "Pneumonia"]

def binarize_labels(df, labels):
    for label in labels:
        df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
    return df

train_val_df = binarize_labels(train_val_df, labels)
test_df = binarize_labels(test_df, labels)

train_val_df = train_val_df[train_val_df["No Finding"] == 0]
train_val_df.drop(columns=["No Finding"], inplace=True)

for label in labels[1:]:
    count = train_val_df[label].sum()
    print(f'Registros de {label}: {count}')

def balance_classes(df, labels):
    min_count = min(df[labels].sum())
    balanced_df = pd.concat([df[df[label] == 1].sample(min_count, random_state=42) for label in labels])
    return balanced_df

train_val_df = balance_classes(train_val_df, labels[1:])

for label in labels[1:]:
    count = train_val_df[label].sum()
    print(f'Registros balanceados de {label}: {count}')

Archivos de prueba y entrenamiento/validación creados exitosamente.


C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Registros de Infiltration: 13782
Registros de Atelectasis: 8280
Registros de Effusion: 8659
Registros de Nodule: 4708
Registros de Pneumothorax: 2637
Registros de Mass: 4034
Registros de Consolidation: 2852
Registros de Pleural_Thickening: 2242
Registros de Cardiomegaly: 1707
Registros de Emphysema: 1423
Registros de Fibrosis: 1251
Registros de Edema: 1378
Registros de Pneumonia: 876
Registros balanceados de Infiltration: 3400
Registros balanceados de Atelectasis: 2567
Registros balanceados de Effusion: 3043
Registros balanceados de Nodule: 1691
Registros balanceados de Pneumothorax: 1381
Registros balanceados de Mass: 1675
Registros balanceados de Consolidation: 1536
Registros balanceados de Pleural_Thickening: 1415
Registros balanceados de Cardiomegaly: 1133
Registros balanceados de Emphysema: 1163
Registros balanceados de Fibrosis: 1088
Registros balanceados de Edema: 1240
Registros balanceados de Pneumonia: 1098


C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
C:\Users\enman\AppData\Local\Temp\ipykernel_11308\2646195574.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [43]:
import numpy as np
import pandas as pd
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import backend as K

labels = ["Infiltration", "Atelectasis", "Effusion", "Nodule", "Pneumothorax", "Mass", "Consolidation", "Pleural_Thickening", "Cardiomegaly", "Emphysema", "Fibrosis", "Edema", "Pneumonia"]

def compute_class_freqs(labels):
    N = labels.shape[0]
    positive_frequencies = np.sum(labels, axis=0) / N
    negative_frequencies = 1 - positive_frequencies
    return positive_frequencies, negative_frequencies

def binarize_labels(df, labels):
    for label in labels:
        df[label] = df['Finding Labels'].apply(lambda x: 1 if label in x else 0)
    return df

def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    def weighted_loss(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx()) 
        loss = 0.0
        for i in range(len(pos_weights)):
            loss_pos = -1 * K.mean(pos_weights[i] * y_true[:, i] * K.log(y_pred[:, i] + epsilon))
            loss_neg = -1 * K.mean(neg_weights[i] * (1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))
            loss += loss_pos + loss_neg
        return loss
    return weighted_loss


train_val_df = pd.read_csv("Documentacion/train_val_balanced.csv")
test_df = pd.read_csv("Documentacion/test_balanced.csv")

train_val_df = binarize_labels(train_val_df, labels)
test_df = binarize_labels(test_df, labels)

pos_weights, neg_weights = compute_class_freqs(train_val_df[labels].values)

datagen = ImageDataGenerator(
    rescale=1./255.,
    samplewise_center=True, 
    samplewise_std_normalization=True, 
    horizontal_flip=False,
    vertical_flip=False, 
    height_shift_range=0.05, 
    width_shift_range=0.1, 
    rotation_range=5, 
    shear_range=0.1,
    fill_mode='reflect',
    zoom_range=0.15
)

train_gen = datagen.flow_from_dataframe(
    dataframe=train_val_df,
    directory=None,
    x_col='Image Path',
    y_col=labels,
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224)
)

valid_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='Image Path',
    y_col=labels,
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size=(224, 224)
)

base_model = DenseNet121(weights='densenet.hdf5', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001), loss=get_weighted_loss(pos_weights, neg_weights), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)

history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.n // train_gen.batch_size,
    validation_data=valid_gen,
    validation_steps=valid_gen.n // valid_gen.batch_size,
    epochs=300,
    callbacks=[early_stopping, reduce_lr]
)

model_save_path = 'DenseNet121_balanced.h5'
model.save(model_save_path)

Found 1274 validated image filenames.
Found 1144 validated image filenames.
Epoch 1/300
79/79 [==============================] - 49s 518ms/step - loss: 0.8864 - accuracy: 0.1017 - val_loss: 1.3730 - val_accuracy: 0.0757 - lr: 0.0010
Epoch 2/300
79/79 [==============================] - 40s 502ms/step - loss: 0.4519 - accuracy: 0.1375 - val_loss: 0.4721 - val_accuracy: 0.1259 - lr: 0.0010
Epoch 3/300
79/79 [==============================] - 40s 502ms/step - loss: 0.4477 - accuracy: 0.1701 - val_loss: 0.4696 - val_accuracy: 0.1347 - lr: 0.0010
Epoch 4/300
79/79 [==============================] - 40s 504ms/step - loss: 0.4410 - accuracy: 0.1971 - val_loss: 0.5155 - val_accuracy: 0.1285 - lr: 0.0010
Epoch 5/300
79/79 [==============================] - 41s 514ms/step - loss: 0.4374 - accuracy: 0.2361 - val_loss: 0.4682 - val_accuracy: 0.1303 - lr: 0.0010
Epoch 6/300
79/79 [==============================] - 43s 544ms/step - loss: 0.4286 - accuracy: 0.2448 - val_loss: 0.6009 - val_accuracy: 0.